In [ ]:
import pandas as pd
import re
import json
import os
from google.colab import drive

# 1. Setup Paths
drive.mount('/content/drive', force_remount=True)
NOTEBOOK_DIR = '/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT'
INPUT_FILE = os.path.join(NOTEBOOK_DIR, 'moroccan_corpus.jsonl')
OUTPUT_FILE = os.path.join(NOTEBOOK_DIR, '50k_sentences.json')

Mounted at /content/drive


In [ ]:
# 2. Configuration
TARGET_COUNT = 50000
collected_sentences = []

print("Starting extraction...")

# 3. Read the file in chunks to save memory (100 rows per chunk)
# This is safer for 700MB files than reading the whole thing at once.
chunk_size = 100
reader = pd.read_json(INPUT_FILE, lines=True, chunksize=chunk_size)

Starting extraction...


In [ ]:
found_enough = False

for chunk_idx, chunk in enumerate(reader):
    if found_enough:
        break

    print(f"Processing chunk {chunk_idx + 1}...")

    # Iterate through rows in this chunk
    for index, row in chunk.iterrows():
        text = row['clean_text']

        # Your specific splitting logic
        if isinstance(text, str): # Ensure text is not null
            raw_splits = re.split(r'[.!?،\n]', text)
            # Clean and filter empty strings
            clean_sents = [s.strip() for s in raw_splits if s.strip()]

            collected_sentences.extend(clean_sents)

        # Check if we reached the target
        if len(collected_sentences) >= TARGET_COUNT:
            # Cut off any extras so we have exactly 50k
            collected_sentences = collected_sentences[:TARGET_COUNT]
            found_enough = True
            break


Processing chunk 1...
Processing chunk 2...


In [ ]:
# 4. Save to a single JSON file
print(f"\nExtraction complete. Collected {len(collected_sentences)} sentences.")
print(f"Saving to {OUTPUT_FILE}...")

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(collected_sentences, f, ensure_ascii=False, indent=4)

print("Done! File saved.")


Extraction complete. Collected 50000 sentences.
Saving to /content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/50k_sentences.json...
Done! File saved.


In [ ]:
import torch
from transformers import pipeline
from tqdm import tqdm
INPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/50k_sentences.json"
OUTPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/labeled_dataset_7emotions_shuffeled.jsonl"
EMOTIONS = ["Joy", "Love", "Surprise", "Sadness", "Anger", "Fear", "Neutral"]
SEED = 42 # The secret to "Resume" working with Shuffle

In [ ]:
# 1. Load the Teacher Model (Atlas-Chat-9B)
print("Loading Atlas-Chat-9B...")
pipe = pipeline(
    "text-generation",
    model="MBZUAI-Paris/Atlas-Chat-9B",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
    return_full_text=False
)

Loading Atlas-Chat-9B...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Device set to use cuda


In [ ]:
# 2. Load & SHUFFLE Data
print(f"📂 Loading data from {INPUT_FILE}...")
with open(INPUT_FILE, "r") as f:
    all_sentences = json.load(f)

print(f"🎲 Shuffling {len(all_sentences)} sentences (Deterministically)...")
random.seed(SEED) # <--- CRITICAL: Ensures order is same every time we restart
random.shuffle(all_sentences)

📂 Loading data from /content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/50k_sentences.json...
🎲 Shuffling 50000 sentences (Deterministically)...


In [ ]:
# 3. Resume Logic
processed_count = 0
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "r") as f:
        processed_count = sum(1 for line in f)
    print(f"♻️ Resuming from shuffled index #{processed_count}")
else:
    print("🆕 Starting fresh with shuffled data.")

🆕 Starting fresh with shuffled data.


In [ ]:
# 4. Streamlined Prompt (No Reason, Just Label)
def get_label(text):
    prompt = [
        {"role": "user", "content": f"""
        Act as a literary critic for a Moroccan Darija novel.
        Analyze the underlying EMOTIONAL TONE of this text, not just the literal words.

        Text: "{text}"

        Rules:
        1. If the text describes romance, longing, or attraction, label it "Love".
        2. If the text describes physical pleasure or fun, label it "Joy".
        3. Only use "Neutral" for purely factual descriptions with no emotion.

        Classify it into EXACTLY one of these 7 categories: {EMOTIONS}

        Return a JSON object with the key "label" only.
        """}
    ]

    try:
        # Reduced max_new_tokens to 15 (faster!) since we only need one word
        outputs = pipe(prompt, max_new_tokens=15, do_sample=False)
        response_text = outputs[0]['generated_text']

        # Extract JSON
        start = response_text.find('{')
        end = response_text.rfind('}') + 1
        data = json.loads(response_text[start:end])

        if data['label'] not in EMOTIONS: return "Neutral"
        return data['label']
    except:
        return None

In [ ]:
remaining = len(all_sentences) - processed_count
print(f"🚀 Processing {remaining} sentences...")

# We skip the first 'processed_count' items in the SHUFFLED list
for text in tqdm(all_sentences[processed_count:]):
    label = get_label(text)

    if label:
        entry = {"sentence": text, "label": label}
        with open(OUTPUT_FILE, "a") as f:
            json.dump(entry, f)
            f.write("\n")

🚀 Processing 50000 sentences...


 44%|████▍     | 21955/50000 [3:52:55<4:39:50,  1.67it/s]

In [ ]:
len(open(OUTPUT_FILE).readlines())

16345

# The "Therapist Generator" Script

In [ ]:
import json
import os
import torch
from transformers import pipeline
from tqdm import tqdm
import random

# --- CONFIGURATION ---
# Path to your 16k file (Upload it or point to Drive)
INPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/labeled_dataset_7emotions_shuffeled.jsonl"
# Output file (Safety first!)
OUTPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/psychologist_dataset_ready.jsonl"

# We focus on emotions that need "Therapy"
# We skip "Neutral" and maybe "Joy" (unless you want a cheerleader bot)
TARGET_EMOTIONS = ["Sadness", "Fear", "Anger", "Love", "Surprise"]
# ---------------------
QUOTAS = {
    "Sadness": 2,
    "Fear": 2,
    "Anger": 2,
    "Love": 2,
    "Surprise": 1
}

In [ ]:
# 1. Load Model (Teacher)
try:
    if pipe: print("✅ Model already loaded.")
except NameError:
    print("⏳ Loading Atlas-Chat-9B...")
    pipe = pipeline(
        "text-generation",
        model="MBZUAI-Paris/Atlas-Chat-9B",
        model_kwargs={"torch_dtype": torch.bfloat16},
        device="cuda",
        return_full_text=False
    )

⏳ Loading Atlas-Chat-9B...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
# 2. The "Treasure Hunt" (Find specific emotions)
print("🔎 Hunting for diverse emotions in your file...")
found_candidates = []
# Create a tracker for how many we found: {"Sadness": 0, "Fear": 0...}
counts = {k: 0 for k in QUOTAS.keys()}

with open(INPUT_FILE, "r") as f:
    for line in f:
        try:
            data = json.loads(line)
            label = data["label"]

            # If this is an emotion we want, and we haven't hit the limit yet
            if label in QUOTAS and counts[label] < QUOTAS[label]:
                found_candidates.append(data)
                counts[label] += 1

            # Stop if we found everything
            if all(counts[k] == QUOTAS[k] for k in counts):
                break
        except:
            continue

print(f"✅ Found {len(found_candidates)} test sentences:")
print(counts)

🔎 Hunting for diverse emotions in your file...
✅ Found 9 test sentences:
{'Sadness': 2, 'Fear': 2, 'Anger': 2, 'Love': 2, 'Surprise': 1}


In [ ]:
# # 3. Resume Logic
# processed_count = 0
# if os.path.exists(OUTPUT_FILE):
#     with open(OUTPUT_FILE, "r") as f:
#         processed_count = sum(1 for line in f)
#     print(f"♻️ Resuming from #{processed_count}")

In [ ]:
def generate_dynamic_therapy(patient_text, emotion):
    # Randomly pick a style
    style = random.choice(["question", "advice", "insight"])

    # Base persona
    base_instruction = f"Act as a professional, empathetic Moroccan psychologist (Doctor). The patient is feeling {emotion}."

    if style == "question":
        task = "Validate their feelings warmly, then ask ONE gentle follow-up question to explore the root cause."
    elif style == "advice":
        task = "Validate their feelings, then Suggest a small, practical coping strategy (e.g., breathing, patience, walking, writing thoughts). Do NOT ask a question."
    else: # insight
        task = "Validate their feelings deeply. Offer a comforting insight or reframe their situation positively. End with a supportive statement. Do NOT ask a question."

    prompt = [
        {"role": "user", "content": f"""
        {base_instruction}
        Patient: "{patient_text}"

        Task: {task}
        Write in natural Moroccan Darija. Keep it short (2-3 sentences).
        """}
    ]

    try:
        outputs = pipe(prompt, max_new_tokens=128, do_sample=True, temperature=0.75)
        return outputs[0]['generated_text'].strip()
    except:
        return None

In [ ]:
# 4. Print Loop
for i, data in enumerate(found_candidates):
    print(f"--- CASE {i+1}: {data['label'].upper()} ---")
    print(f"🗣️  PATIENT: {data['sentence']}")

    reply = generate_dynamic_therapy(data['sentence'], data['label'])

    print(f"👩‍⚕️ DOCTOR: {reply}\n")
    print("-" * 60)

--- CASE 1: LOVE ---
🗣️  PATIENT: ااا جهاد : وعلاش شنو فيها الا بقينا مع بعضيااتنا امم ليل طويل وفصباح نرجعك اوك ردت بتردد كتشوف فيه : واخا صافي رجعو الاوطيل وهوا مزال مبغى يرخي من خصرها شادها بقوة طلعو فالاسنسور حتا لطابق خامس ودخلو لثاني شومبر حتا لاسنسور
👩‍⚕️ DOCTOR: "مزيان بزاف أنك قادر تعيش هاد الحب. شنو اللي خلاك تحس بهاد الحب لهاد الشخص؟"

------------------------------------------------------------
--- CASE 2: ANGER ---
🗣️  PATIENT: عفت بعصبية:لي خاصو يدااار
👩‍⚕️ DOCTOR: "فهمت غضبك. راه طبيعي تحس هاكا فبعض المرات. غير خود نفس عميق وحاول تشوف الأمور بإيجابية - يمكن تكون فرصة باش تحل مشكلتك بطريقة أحسن."

------------------------------------------------------------
--- CASE 3: LOVE ---
🗣️  PATIENT: فيقاتو من سهوتو كلمة سنين مسمعهاش خلاتو يرمي جساس القاسي ويرجع جساس لحنين الماس:كنبغيك
👩‍⚕️ DOCTOR: هاد الكلمات كيقدرو يكونو قويين بزاف، مزيان أنك عشتي هاد المشاعر. شنو اللي خلاك تحس بهاد الحب تجاه هاد الشخص؟

------------------------------------------------------------
--- CASE 4: ANG

# The data our QLoRA model will be trained on :

In [ ]:
import json
import os
import torch
from transformers import pipeline
from tqdm import tqdm
import random

# --- CONFIGURATION ---
INPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/labeled_dataset_7emotions_shuffeled.jsonl"
OUTPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/psychologist_dataset_ready.jsonl"

# 🧠 SMART BALANCING STRATEGY
# We want MAXIMUM data for problems, but LIMITED data for happy stuff.
TARGET_LIMITS = {
    "Sadness": 5000,   # Take as much as possible
    "Fear": 5000,      # Take as much as possible
    "Anger": 5000,     # Take as much as possible
    "Love": 500,      # CAP THIS! Don't let it drown out the others.
    "Surprise": 500,   # Keep it low
    "Joy": 500         # Keep it low (unless you want a cheerleader)
}
# ---------------------

In [ ]:
# 1. Load Model
try:
    if pipe: print("✅ Model already loaded.")
except NameError:
    print("⏳ Loading Atlas-Chat-9B...")
    pipe = pipeline(
        "text-generation",
        model="MBZUAI-Paris/Atlas-Chat-9B",
        model_kwargs={"torch_dtype": torch.bfloat16},
        device="cuda",
        return_full_text=False
    )

✅ Model already loaded.


In [ ]:
# 2. Filter & Balance the Data
print("⚖️ Balancing the dataset...")
candidates = []
counts = {k: 0 for k in TARGET_LIMITS.keys()}

# We read the whole file first to shuffle it (so we don't just get the first 1500 Love sentences)
all_lines = open(INPUT_FILE, "r").readlines()
random.shuffle(all_lines)

for line in all_lines:
    try:
        data = json.loads(line)
        label = data["label"]

        # If valid emotion AND we haven't hit the cap yet
        if label in TARGET_LIMITS and counts[label] < TARGET_LIMITS[label]:
            candidates.append(data)
            counts[label] += 1
    except:
        continue

print(f"🎯 Final Dataset Mix: {counts}")
print(f"🚀 Generating responses for {len(candidates)} patients...")

⚖️ Balancing the dataset...
🎯 Final Dataset Mix: {'Sadness': 2470, 'Fear': 709, 'Anger': 1057, 'Love': 500, 'Surprise': 365, 'Joy': 500}
🚀 Generating responses for 5601 patients...


In [ ]:
# 3. Resume Logic
processed_count = 0
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "r") as f:
        processed_count = sum(1 for line in f)
    print(f"♻️ Resuming from #{processed_count}")

In [ ]:
# 4. The Doctor Prompt
def generate_dynamic_therapy(patient_text, emotion):
    # Randomly pick a style
    style = random.choice(["question", "advice", "insight"])

    # Base persona
    base_instruction = f"Act as a professional, empathetic Moroccan psychologist (Doctor). The patient is feeling {emotion}."

    if style == "question":
        task = "Validate their feelings warmly, then ask ONE gentle follow-up question to explore the root cause."
    elif style == "advice":
        task = "Validate their feelings, then Suggest a small, practical coping strategy (e.g., breathing, patience, walking, writing thoughts). Do NOT ask a question."
    else: # insight
        task = "Validate their feelings deeply. Offer a comforting insight or reframe their situation positively. End with a supportive statement. Do NOT ask a question."

    prompt = [
        {"role": "user", "content": f"""
        {base_instruction}
        Patient: "{patient_text}"

        Task: {task}
        Write in natural Moroccan Darija. Keep it short (2-3 sentences).
        """}
    ]

    try:
        outputs = pipe(prompt, max_new_tokens=128, do_sample=True, temperature=0.75)
        return outputs[0]['generated_text'].strip()
    except:
        return None

In [ ]:
print(f"🚀 Generating 'Smart' responses for {len(candidates) - processed_count} patients...")

for data in tqdm(candidates[processed_count:]):
    patient_text = data["sentence"]
    emotion = data["label"]

    # Use the new function
    doctor_reply = generate_dynamic_therapy(patient_text, emotion)

    if doctor_reply:
        entry = {
            "instruction": "Act as a Moroccan psychologist and help this user.",
            "input": patient_text,
            "output": doctor_reply,
            "emotion_tag": emotion
        }

        with open(OUTPUT_FILE, "a") as f:
            json.dump(entry, f, ensure_ascii=False)
            f.write("\n")

🚀 Generating 'Smart' responses for 5601 patients...


100%|██████████| 5601/5601 [6:21:41<00:00,  4.09s/it]


# The Golden Model

In [ ]:
# 1. Install Training Libraries
!pip install -q -U torch bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers
!pip install -q -U git+https://github.com/huggingface/peft
!pip install -q -U git+https://github.com/huggingface/trl

import torch
import os
import shutil
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer

# --- CONFIGURATION ---
DATASET_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/psychologist_dataset_ready.jsonl"
MODEL_ID = "MBZUAI-Paris/Atlas-Chat-9B"
NEW_MODEL_NAME = "Atlas-Psychologist-Darija-v1"
DRIVE_BACKUP_PATH = f"/content/drive/MyDrive/{NEW_MODEL_NAME}"
# ---------------------

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 154.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170

KeyboardInterrupt: 